# Нормализация, секцияция и чанкинг Markdown

В этом ноутбуке разбиваем Markdown по заголовкам и создаём чанки (~800 токенов, overlap 120). Результат сохраняется в JSONL для последующей векторизации и загрузки в Qdrant.

- Вход: `data/processed/md/{PHB,DMG}.md`
- Выход: `data/processed/chunks/{PHB,DMG}.jsonl`


## Шаг 1. Нормализация Markdown (очистка шума)

Перед чанкингом очищаем Docling/OCR‑артефакты и приводим текст к «готовому к векторизации» виду. Используем объединённый набор правил:

- **Декодирование спецпоследовательностей**: `&amp;` → `&`, `/uniXXXX` → соответствующий символ Юникода.
- **Удаление шумных строк**: сбрасываем короткие OCR-артефакты вроде `@DMG.md (2-11)` или одиночные знаки без букв.
- **Склейка разорванных слов и переносов**: `П оследующие` → `Последующие`, `D UNGEONS` → `DUNGEONS`, `буква-\nбуква` → `буквабуква`, внутрисловной `\n` → пробел, лишние пустые строки схлопываются.
- **Дефис/тире и пробелы**: внутри слов `a - b` → `a-b`; между словами ` - ` → ` — `; лишние пробелы перед знаками препинания убираются.
- **Восстановление буквиц**: при наличии `pymorphy2` восстанавливаем пропавшую первую букву в параграфах с буквицы.
- **Омографы (латиница↔кириллица)**: латинские `A,B,C,E,H,K,M,O,P,T,X,Y` и строчные аналоги переводятся в кириллицу внутри русских слов.
- **Частный артефакт `fа`**: удаляем начальное латинское `f` перед русской гласной (`fа` → `а`).
- **Сдвоенные слова** после склейки строк: `персонажи персонажи` → `персонажи`.
- **Unicode NFC** и финальная чистка пробелов/пустых строк.
- **LLM-постобработка (опция)**: параметры находятся в `configs/ingest.yaml` (секция `llm_postprocess`). При `enabled: true` ищем подозрительные абзацы, отправляем их в OpenAI через LangChain и фиксируем изменения в `logs/llm_cleanup/*.json`. Ключ и модель кладём в `.env` (`OPENAI_API_KEY`, `OPENAI_MODEL`), файл в git не добавляем.

Результат пишем в `data/processed/md_clean/*.md` и используем далее для чанкинга.


In [ ]:
# Нормализация исходных Markdown в отдельную директорию
import sys
from pathlib import Path

def _detect_repo_root() -> Path:
    start = Path.cwd().resolve()
    for p in [start] + list(start.parents):
        if (p / 'src').is_dir() and (p / 'data').is_dir():
            return p
    return start

try:
    # Сработает, если код запущен как скрипт, а не ноутбук
    repo_root = Path(__file__).resolve().parent.parent
except NameError:
    # В ноутбуке __file__ нет — ищем корень по структуре проекта
    repo_root = _detect_repo_root()

sys.path.append(str(repo_root / 'src'))

from dnd_rag.core.pipelines import normalize_md_dir_pipeline  # type: ignore
from dnd_rag.core.config import DEFAULT_CONFIG_PATH  # type: ignore

RAW_MD_DIR = repo_root / 'data' / 'processed' / 'md'
CLEAN_MD_DIR = repo_root / 'data' / 'processed' / 'md_clean'

normalized = normalize_md_dir_pipeline(
    RAW_MD_DIR,
    CLEAN_MD_DIR,
    config_path=DEFAULT_CONFIG_PATH,
)
normalized

## Шаг 2. LLM-постобработка (опционально)

Этап запускается вручную после полной нормализации. Используются переменные окружения `LLM_API_KEY`, `MODEL`, при необходимости `URL_MODEL` (для OpenRouter совместимых провайдеров). Результат — инплейс‑правки в `data/processed/md_clean/*.md` и журнал `logs/llm_cleanup/*.json` (пары before/after).


In [ ]:
# Запуск LLM-постобработки (при необходимости)
import sys
from pathlib import Path

def _detect_repo_root() -> Path:
    start = Path.cwd().resolve()
    for p in [start] + list(start.parents):
        if (p / 'src').is_dir() and (p / 'data').is_dir():
            return p
    return start

try:
    repo_root
except NameError:
    repo_root = _detect_repo_root()

sys.path.append(str(repo_root / 'src'))

from dnd_rag.core.pipelines import llm_cleanup_md_dir_pipeline  # type: ignore
from dnd_rag.core.config import DEFAULT_CONFIG_PATH  # type: ignore

CLEAN_MD_DIR = repo_root / 'data' / 'processed' / 'md_clean'

# Опционально можно переопределить параметры
produced = llm_cleanup_md_dir_pipeline(
    CLEAN_MD_DIR,
    config_path=DEFAULT_CONFIG_PATH,
    # min_score=0.45,
    # max_paragraphs=500,
)
produced


## Шаг 3. Чанкинг очищенного Markdown

Берём файлы из `data/processed/md_clean`, секционируем по заголовкам и режем на перекрывающиеся чанки (~800 токенов, overlap 120). Выход — `data/processed/chunks/*.jsonl` для последующей векторизации/загрузки в Qdrant. Используем `chunk_docs_pipeline` с дефолтной конфигурацией.


In [1]:
# Импорт и запуск пайплайна чанкинга
import sys
from pathlib import Path

def _detect_repo_root() -> Path:
    start = Path.cwd().resolve()
    for p in [start] + list(start.parents):
        if (p / 'src').is_dir() and (p / 'data').is_dir():
            return p
    return start

# Добавляем src/ в PYTHONPATH
try:
    repo_root
except NameError:
    try:
        repo_root = Path(__file__).resolve().parents[1]
    except NameError:
        repo_root = _detect_repo_root()

sys.path.append(str(repo_root / 'src'))

from dnd_rag.core.pipelines import chunk_docs_pipeline #type: ignore
from dnd_rag.core.config import DEFAULT_CONFIG_PATH #type: ignore

IN_MD_DIR = repo_root / 'data' / 'processed' / 'md_clean'
OUT_CH_DIR = repo_root / 'data' / 'processed' / 'chunks'
CONFIG = DEFAULT_CONFIG_PATH

produced = chunk_docs_pipeline(IN_MD_DIR, OUT_CH_DIR, config_path=CONFIG)
produced


c:\Users\Shchurov\AppData\Local\Programs\Python\Python312\Lib\site-packages\pymorphy2\analyzer.py:114: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
2025-11-11 23:52:58,435 - INFO - Loading dictionaries from c:\Users\Shchurov\AppData\Local\Programs\Python\Python312\Lib\site-packages\pymorphy2_dicts_ru\data
2025-11-11 23:52:58,511 - INFO - format: 2.4, revision: 417127, updated: 2020-10-11T15:05:51.070345


[CFG] ingest.yaml: C:\Users\Shchurov\Хранилище\Документы\05_Программирование\02_projects\personal\dnd_rule_assistant\configs\ingest.yaml (exists=True)


[WindowsPath('C:/Users/Shchurov/Хранилище/Документы/05_Программирование/02_projects/personal/dnd_rule_assistant/data/processed/chunks/DMG.jsonl')]

## Мини-поиск (sanity check)

Выполним простой поиск по ключевым словам, чтобы убедиться, что в чанках есть релевантные фрагменты. Для производительного поиска позже будет использован Qdrant + эмбеддинги, здесь — лишь быстрая проверка.


In [ ]:
# Простой keyword-поиск по JSONL
from pathlib import Path
import json

def _detect_repo_root() -> Path:
    start = Path.cwd().resolve()
    for p in [start] + list(start.parents):
        if (p / 'src').is_dir() and (p / 'data').is_dir():
            return p
    return start

try:
    repo_root
except NameError:
    try:
        repo_root = Path(__file__).resolve().parents[1]
    except NameError:
        repo_root = _detect_repo_root()

CH_DIR = repo_root / 'data' / 'processed' / 'chunks'

files = sorted(CH_DIR.glob('*.jsonl'))
rows = []
for fp in files:
    with fp.open('r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            rows.append(json.loads(line))

query = 'что такое преимущество'
terms = [t for t in query.lower().split() if t]

def score(text: str) -> int:
    low = (text or '').lower()
    return sum(low.count(t) for t in terms)

rows.sort(key=lambda r: score(r.get('text', '')), reverse=True)
rows[:5]
